In [1]:
# This file gets OAuth tokens
import requests
import json
import csv
import praw
import re
from praw.models import MoreComments
from tickers_names import *
import yfinance as yf
import nltk
from nltk.corpus import stopwords
import datetime
from datetime import datetime as dt
import time
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# We made the SECRET_KEY and password private before making the repo public.


CLIENT_ID = None
SECRET_KEY = None

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {'grant_type': 'client_credentials',
        'username': 'CS229project',
        'password': None}
headers = {'User-Agent': 'CS229project/0.0.1'}
base_url = 'https://www.reddit.com/'
res = requests.post(base_url + 'api/v1/access_token', auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res_json = res.json()
print(res_json)


{'access_token': '-iDl6sW_szkHvaWF1L_rS7Xrhgb83EA', 'token_type': 'bearer', 'expires_in': 3600, 'scope': '*'}


In [3]:
reddit = praw.Reddit(
    client_id=CLIENT_ID, 
    client_secret=SECRET_KEY, 
    user_agent='CS229project/0.0.1',
    username='CS229project',
    password=None)

In [4]:

TOKEN = res_json['access_token']
# add authorization to our headers dictionary
headers = {'User-Agent': 'CS229project/0.0.1', 'Authorization': f'bearer {TOKEN}', }

In [5]:
# general post data from r/wsb hot 
r = requests.get('https://oauth.reddit.com/r/wallstreetbets/hot', headers=headers)
data = json.loads(r.text)['data']['children']

with open('reddit_data.csv', 'w', encoding='utf8') as f:  
    writer = csv.writer(f)
    for dp in data:
        for key, value in dp['data'].items(): 
            writer.writerow([key, value])
print("reddit_data.csv file written") 

reddit_data.csv file written


In [6]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()
print(res)

{'features': {'mod_service_mute_writes': True, 'promoted_trend_blanks': True, 'show_amp_link': True, 'mweb_in_feed_refresh': {'owner': 'growth', 'variant': 'random', 'experiment_id': 507}, 'is_email_permission_required': False, 'mod_awards': True, 'mweb_xpromo_revamp_v3': {'owner': 'growth', 'variant': 'treatment_1', 'experiment_id': 480}, 'chat_subreddit': True, 'awards_on_streams': True, 'mweb_xpromo_modal_listing_click_daily_dismissible_ios': True, 'modlog_copyright_removal': True, 'do_not_track': True, 'mod_service_mute_reads': True, 'chat_user_settings': True, 'use_pref_account_deployment': True, 'mweb_xpromo_interstitial_comments_ios': True, 'mweb_xpromo_modal_listing_click_daily_dismissible_android': True, 'premium_subscriptions_table': True, 'mweb_xpromo_interstitial_comments_android': True, 'mweb_footer_upsell': {'owner': 'growth', 'variant': 'light_1', 'experiment_id': 497}, 'chat_group_rollout': True, 'resized_styles_images': True, 'spez_modal': True, 'noreferrer_to_noopener

In [102]:
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'features': {'mod_service_mute_writes': True,
  'promoted_trend_blanks': True,
  'show_amp_link': True,
  'is_email_permission_required': False,
  'mod_awards': True,
  'expensive_coins_package': True,
  'mweb_xpromo_revamp_v2': {'owner': 'growth',
   'variant': 'treatment_4',
   'experiment_id': 457},
  'awards_on_streams': True,
  'mweb_xpromo_modal_listing_click_daily_dismissible_ios': True,
  'chat_subreddit': True,
  'modlog_copyright_removal': True,
  'do_not_track': True,
  'mod_service_mute_reads': True,
  'chat_user_settings': True,
  'use_pref_account_deployment': True,
  'mweb_xpromo_interstitial_comments_ios': True,
  'mweb_xpromo_modal_listing_click_daily_dismissible_android': True,
  'premium_subscriptions_table': True,
  'mweb_xpromo_interstitial_comments_android': True,
  'noreferrer_to_noopener': True,
  'mweb_footer_upsell': {'owner': 'growth',
   'variant': 'control_1',
   'experiment_id': 497},
  'chat_group_rollout': True,
  'resized_styles_images': True,
  'spez_

In [5]:
with open('urls.csv', 'r', encoding='utf8') as f:
    reader = csv.reader(f)
    urls = []
    for row in reader:
        urls.append(row[0])

In [6]:
#urls = ["https://www.reddit.com/r/wallstreetbets/comments/nm414o/daily_discussion_thread_for_may_27_2021/"]
for url in urls[2:70]:
    comments = reddit.submission(url=url).comments
    comments.replace_more(limit=32)
    i = 0
    t_list, t_dict = convert_data()
    stop_words = set(stopwords.words('english'))
    # add_words = ('one', 'all', 'fang', 'dd', 'elon', 'time', 'long', 'call', 'ever', 'see', 'gain', 'pm', 'rent', 'omg', 'baby', 'data', 'beat', 'win', 'tiny', 'tech', 'ago', 'type', 'bod', 'eod', 'man', 'else', 'true', 'save', 'away', 'min', 'max', 'news', 'n', 'look', 'news', 'beat', 'rent', 'good', 'pre', 'rate', 'club', 'hill', 'tax', 'fds', 'sale', 'per', 'full', 'ev', 'per', 'rice', 'men', 'king', 'post', 'well', 'town', 'r', 'cdc', 'su', 'af', 'line', 'dds', 'eat', 'well', 'free', 'nice', 'cool', 'plus', 'bit', 'bro', 'sale', 'date', 'mr', 'king', 'gg', 'pay', 'per', 'fish', 'flat', 'rate', 'semi', 'game', 'im', 'kang', 'act', 'via', 'avg', 'jfc', 'sir', 'lock', 'core', 'hear', 'fun', 'fast', 'big', 'rock', 'cash', 'air', 'gold', 'hr', 'car', 'trip', 'door', 'ape', 'hot', 'code', 'sons', 'fuel', 'mini', 'name', 'play', 'calm', 'two', 'jobs', 'glad', 'cost', 'tho', 'mo', 'luv', 'ship', 'fly', 'ppl', 'mind', 'glad', 'zen', 'mark', 'blue', 'leaf', 'name', 'low', 'wood', 'jazz', 'cuz', 'k', 'stay', 'wash', 'earn', 'ed', 'ego', 'farm', 'flex', 'fold', 'form', 'fuel', 'soul', 'star', 'stay', 'tap', 'good', 'hear', 'heat', 'hero', 'herb', 'hi', 'hive', 'hog', 'huge', 'ice', 'idea', 'if', 'it', 'jack', 'job', 'jobs', 'joy', 'keg', 'key', 'keys', 'lad', 'leg', 'lime', 'lock', 'look', 'ma', 'mark', 'meet', 'meme', 'mil', 'mill', 'more', 'move', 'my', 'nice', 'nine', 'oink', 'one', 'peg', 'plow', 'plus', 'pope', 'pro', 'pool', 'punk', 'pure', 'push', 'rare', 'rate', 'rent', 'roll', 'rick', 'salt', 'sand', 'see', 'semi', 'ship', 'sky', 'slot', 'snow', 'soap', 'soda', 'sons', 'tier', 'tiny', 'tier', 'tops', 'town', 'tree', 'vest', 'walk', 'wash', 'warm', 'wave', 'wild', 'wrap', 'beer', 'elly', 'bid', 'bio', 'bit', 'born', 'bossy', 'bud', 'cab', 'call', 'carb', 'cart', 'cry', 'chef', 'earns', 'earn', 'ego', 'exam', 'gas') 
    #for word in add_words:
    #    stop_words.add(word)
    # comments.replace_more(limit=0)
    url_date = url.split('/')[-2].split('_')
    with open('reddit_data/reddit_data_' + url_date[-3] + url_date[-2] + url_date[-1] + '.csv', 'w', encoding='utf8') as f:  
        writer = csv.writer(f)
        for comment in comments:
            #print(comment.body.lower())
            # extract mentioned stocks
            ticker_list = []
            words = []
            score = comment.score
            for word in comment.body.lower().split():
                words.append(" ".join(re.findall("[a-zA-Z]+", word)))
            # print("####\n", words)
            for word in words:
                if (word not in ticker_list) and (word not in stop_words) and (word in t_list):
                    ticker_list.append(word)
                elif word in t_dict and t_dict[word] not in ticker_list and word not in stop_words:
                    ticker_list.append(t_dict[word])
            # get percent change, write rows
            next_day = str(dt.fromtimestamp(comment.created_utc + 86400)).split()[0]
            seven_days_back = str(dt.fromtimestamp(comment.created_utc - 86400 * 6)).split()[0]
            seven_days_forward = str(dt.fromtimestamp(comment.created_utc + 86400 * 8)).split()[0]
            for ticker in ticker_list:
                try:  
                    df = yf.Ticker(ticker).history(start=seven_days_back, end=seven_days_forward, interval="1d")
                    percent_change_back = df.iloc[4][3] / df.iloc[0][0]
                    percent_change_forward = df.iloc[-1][3] / df.iloc[4][0]
                    i += 1
                    writer.writerow([words, ticker, score, percent_change_back, percent_change_forward])
                except:
                    pass
            if i >= 1000:
                break
    print(url_date[-3] + url_date[-2] + url_date[-1] + "done, ", "data points written: ", i)

- INB: No data found, symbol may be delisted
- VR: No data found for this date range, symbol may be delisted
- PE: No data found, symbol may be delisted
- ROSE: No data found, symbol may be delisted
- Q: No data found for this date range, symbol may be delisted
- SPAN: No data found for this date range, symbol may be delisted
- FTD: No data found, symbol may be delisted
- P: No data found for this date range, symbol may be delisted
-  RKT: No data found, symbol may be delisted
- P: No data found for this date range, symbol may be delisted
-  RKT: No data found, symbol may be delisted
- MM: No data found for this date range, symbol may be delisted
- P: No data found for this date range, symbol may be delisted
- PE: No data found, symbol may be delisted
- P: No data found for this date range, symbol may be delisted
- P: No data found for this date range, symbol may be delisted
- WAGE: No data found, symbol may be delisted
-  RKT: No data found, symbol may be delisted
- P: No data found f

In [55]:
urls = ["https://www.reddit.com/r/wallstreetbets/comments/nc4z12/daily_discussion_thread_for_may_14_2021/"]
for url in urls:
    comments = reddit.submission(url=url).comments
    print(comments[0].score)

118


In [36]:
url = "https://www.reddit.com/r/wallstreetbets/comments/nc4z12/daily_discussion_thread_for_may_14_2021/"
comment = reddit.submission(url=url).comments[0]
print(comment.created_utc)

1620988034.0
